# AIFFEL LMS Exploration 11 project - 멋진 작사가 만들기



## Step 1. 데이터 다운로드

## Step 2. 데이터 읽어오기

In [1]:
# 패키지 호출
import re     # 정규표현식 지원
import glob   # 여러개 파일명을 리스트로 받는 glob 함수 포함
import os     # 파일 경로 관련
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# 경로 설정
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
# glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환

raw_corpus = []

# 여러 개의 txt 파일을 모두 읽어서 raw_corpus에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        # 줄마다 분리
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
## 질문 이렇게 추가하면 한 노래 마지막과 다음 노래 처음도 리스트로 이어지는데, 학습에 영향을 미치지 않나?

print("데이터 크기 :", len(raw_corpus))
print("Example: \n", raw_corpus[:3])    # 예시로 3개만 뽑기
        

데이터 크기 : 187088
Example: 
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


## Step 3. 데이터 정제

 * prepocess_sentence() 함수 만든 것 이용
 
 * 지나치게 긴 문장은 과도한 padding을 유발하므로 삭제할 것
 
 * 문장을 토큰화 했을 때, 토큰 갯수가 15개를 넘어가면 삭제 처리

 * 전처리 함수 정의 
    - 특수문자, 2개 이상 공백 삭제
    - < start >, < end > 추가

In [3]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()    # 소문자로 바꾸고 양쪽 공백을 삭제
    
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r"\1", sentence) # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백 추가
    sentence = re.sub(r'[""]+', " ", sentence)    # 공백패턴을 만나면, 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()    # 문자열 양 끝에 공백, \n을 삭제해줌
    
    # <start> 뒤와 <end> 앞에 띄어쓰기 있는 것 주의!!
    sentence = '<start> ' + sentence + ' <end>'    # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여줌
    
    return sentence

 * 전처리 함수 수행

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.    
    corpus.append(preprocess_sentence(sentence)) # raw 데이터에서 전처리 된 문장들면 추가
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr. seuss <end>',
 '<start> the sun did not shine. <end>',
 '<start> it was too wet to play. <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day. <end>',
 '<start> i sat there with sally. <end>',
 '<start> we sat there we two. <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do! <end>']

 * 토큰화 함수 정의

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 15000,    # 전체 단어의 갯수
        filters = '', # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token = "<unk>"   # out-of-vocabulary : 사전에 없는 단어를 unk로 대체
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동 구축
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축
    tensor = tokenizer.texts_to_sequences(corpus) # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환
    
    # padding 작업 수행
    # maxlen = 15(최대 길이를 15로 조정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen = 15, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    5 1155 ...    0    0    0]
 [   2  119 4844 ...    0    0    0]
 [   2    5  335 ...    0    0    0]
 ...
 [ 699   26    5 ...    5  205    3]
 [   2  699   26 ...    0    0    0]
 [   2  699   26 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe354b4ddd0>


In [6]:
# 함수 확인
# tf.keras.preprocessing.sequence.pad_sequences??

## Step 4. 평가 데이터셋 분리

 * sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리
 *  단어장의 크기는 12,000 이상으로 설정
 * 총 데이터의 20%를 평가 데이터셋으로 사용

 *  데이터 확인

In [7]:
# type(tensor)

numpy.ndarray

In [8]:
# tensor[:3]

array([[    2,     5,  1155,    13,     5,  1677,     3,     0,     0,
            0,     0,     0,     0,     0,     0],
       [    2,   119,  4844,     1,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [    2,     5,   335,   167,    63, 11413,     3,     0,     0,
            0,     0,     0,     0,     0,     0]], dtype=int32)

In [9]:
# tensor.shape

(175749, 15)

In [10]:
train = tensor[:, :-1]   # 마지막 값(end 또는 padding) 제거
val = tensor[:, 1:]     # 시작 값 <start>=2 제거

# enc : encoder, dec : decoder
enc_train, enc_val, dec_train, dec_val = train_test_split(train, val, test_size=0.2, random_state = 1)

In [11]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## Step 5. 인공지능 만들기
    * 모델의 Embedding Size와 Hidden Size를 조절하며 
      10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!
      (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

    * 그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [12]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1
# tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
    
        return out

embedding_size = 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 73s 134ms/step - loss: 3.7565
Epoch 2/10
549/549 [==============================] - 73s 132ms/step - loss: 3.2698
Epoch 3/10
549/549 [==============================] - 73s 133ms/step - loss: 3.0908
Epoch 4/10
549/549 [==============================] - 73s 133ms/step - loss: 2.9558
Epoch 5/10
549/549 [==============================] - 73s 133ms/step - loss: 2.8435
Epoch 6/10
549/549 [==============================] - 73s 133ms/step - loss: 2.7439
Epoch 7/10
549/549 [==============================] - 73s 133ms/step - loss: 2.6535
Epoch 8/10
549/549 [==============================] - 73s 133ms/step - loss: 2.5700
Epoch 9/10
549/549 [==============================] - 73s 133ms/step - loss: 2.4910
Epoch 10/10
549/549 [==============================] - 73s 133ms/step - loss: 2.4156


In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

# 느낀점 및 결론

 * 문자 데이터 전처리에 사용하는 정규표현식 사용방법이 직관적이지 않아 어려웠다. 추후에 따로 정리 및 학습이 필요할 것 같다
